<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/recife_bom_de_bola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='30%' align='Center' src='https://github.com/leonarddocchagas/data_science/blob/master/projetos/recife_bom_de_bola/logo_recife_bom_de_bola_2014.png?raw=true' />

Maior campeonato de futebol de campos de várzea do mundo, o Recife Bom de Bola promove a inclusão social e a integração entre as diversas comunidades do Recife através do Esporte. Realizado pela Prefeitura do Recife, por meio da Secretaria de Esportes do Recife (SESP), o torneio envolve no ano de 2021, cinco categorias e mais de 350 times.

Além da interação social, o Recife Bom de Bola também vai garantir curso de arbitragem para representantes das comunidades e para professores vinculados à SESP. Ocorrerá ainda o Fórum Bom de Bola, para discussões diversas e benefícios para os destaques da categoria sub-15, que serão encaminhados para avaliações nos principais clubes do Recife.

<i>Fonte: https://recifebomdebola.com.br/o-projeto/<i/>

##Sobre os dados

Vamos trabalhar com 3 conjuntos de dados e todos vindo do http://dados.recife.pe.gov.br/.

* Dataset "campos": https://bit.ly/3n1kXg6
  * Dicionário: https://bit.ly/3kHPTiM
  * Variáveis:
    * rpa
    * endereco
    * bairro
    * latitude
    * longitude
    * privado
    * nome

* Dataset "Equipes": https://bit.ly/3gY6M7M
  * Dicionário: https://bit.ly/3gUhrAw
  * Variáveis:
    * rpa
    * categoria
    * equipe
    * cartao_amarelo
    * cartao_vermelho

* Dataset "Jogadores": https://bit.ly/3jFe0iF
  * Dicionário: https://bit.ly/3kHQBMY
  * Variáveis:
    * rpa_equipe
    * modalidade_equipe
    * nome_equipe
    * representante_equipe
    * tecnico
    * nome_jogador
    * numero_jogador
    * bairro_jogador
    * cidade_jogador
    * dtnascimento_jogador

##Obteção e transformação dos dados

**Carrega e verifica dataset**

In [1]:
# bibliotecas
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime

In [14]:
df_equipes = 'http://dados.recife.pe.gov.br/dataset/a253e40a-ccb2-474e-902f-513deb3d7614/resource/3838af9e-2c53-4cad-9716-e6ca76f1908f/download/equipes-participantes-recife-bom-de-bola.csv'
df_equipes = pd.read_csv(df_equipes)

URLError: ignored

In [10]:
# import dataset equipes
df_equipes = 'http://dados.recife.pe.gov.br/dataset/a253e40a-ccb2-474e-902f-513deb3d7614/resource/3838af9e-2c53-4cad-9716-e6ca76f1908f/download/equipes-participantes-recife-bom-de-bola.csv'
df_equipes = pd.read_csv(df_equipes, sep=';')
df_equipes.head()

URLError: ignored

In [11]:
# import dataset campos
df_campos = 'http://dados.recife.pe.gov.br/dataset/a253e40a-ccb2-474e-902f-513deb3d7614/resource/adbc9234-b5cc-4616-9372-86858a6f3d13/download/campos-recife-bom-de-bola.csv'
df_campos = pd.read_csv(df_campos, sep=';', error_bad_lines=False)
df_campos = df_campos.drop(columns='enderco')
df_campos.head()

URLError: ignored

In [4]:
# import dataset jogadores
df_jogadores = 'http://dados.recife.pe.gov.br/dataset/a253e40a-ccb2-474e-902f-513deb3d7614/resource/d1121f92-bae2-4702-ae71-d844ae2a0120/download/dados-dos-jogadores.csv'
df_jogadores = pd.read_csv(df_jogadores, error_bad_lines=False, encoding='iso-8859-1', sep=';')
df_jogadores = df_jogadores.drop(columns=['REPRESENTANTE_EQUIPE', 'TECNICO'])
df_jogadores.head()

URLError: ignored

Identificamos que em nosso conjunto de dados **df_jogadores**, existem registros na coluna `DTNASCIMENTO_JOGADOR` que não segue o padrão. Vamos removelo para converter essa coluna num tipo date.

In [ ]:
# identificando registros fora do padrão
df_jogadores['Teste'] = df_jogadores['DTNASCIMENTO_JOGADOR'].apply(lambda x: x.find('--') if 'tem' else 'nao')

In [ ]:
# exibindo registros fora do padrão
df_jogadores.loc[df_jogadores['Teste'] != -1]


Foram encontrados *168* fora do padrão. Vamos deletar e seguir com as análises.

In [ ]:
# deletando registros fora do padrão de data
df_jogadores = df_jogadores.drop(df_jogadores.loc[df_jogadores['Teste'] != -1].index)
df_jogadores = df_jogadores.drop(columns='Teste')

# exibi dataset tratado
df_jogadores.head()

In [ ]:
# convertindo coluna data para tipo date
df_jogadores['DTNASCIMENTO_JOGADOR'] = pd.to_datetime(df_jogadores['DTNASCIMENTO_JOGADOR'])

**Revisando os 3 datasets**

In [ ]:
# dataset campo
df_campos.head()

In [ ]:
# dataset jogadores
df_jogadores.head()

In [ ]:
# dataset equipes
df_equipes.head()

In [ ]:
# df_equipes['equipe'] = df_equipes['equipe'].str.title()

## Visualizar 

In [ ]:
# importa biblioteca para mapas
import folium
from folium import plugins

**Dataset: Campo**

In [ ]:
# exibi as 5 primeiras entradas
df_campos.head()

In [ ]:
# Total de 22 campos
df_campos['nome'].count()

In [ ]:
# eliminando espaços entre os nomes
df_campos['bairro'] = df_campos['bairro'].str.strip()

In [ ]:
# total de campos por bairro
df_campos['bairro'].value_counts()

In [ ]:
# plotando mapa
mapa = folium.Map(
    location = [-8.0704276,-34.894891],
    zoom_start=12,
    tiles='OpenStreetMap'
)

# criando pontos
for i, row in df_campos.iterrows(): 
  latitude = df_campos.at[i, 'latitude']
  longitude = df_campos.at[i, 'longitude']
  bairro = df_campos.at[i, 'bairro']

  folium.Marker(
      location = [latitude, longitude],
      popup=bairro,
      icon=folium.Icon('blue')
  ).add_to(mapa)

# exibe mapa
mapa

In [ ]:
# plotando gráfico
coluna = "bairro"
grafico = px.bar(data_frame=df_campos, y=coluna,  orientation='h')
grafico.show()

**Dataset: Jogadores**

In [ ]:
# dataset jogadores
df_jogadores.head()